In [1]:
from src.utils import read_ml_1m
from src.utils import split_recsys_data
from src.utils import split_by_user_folds

In [2]:
ratings, item_data, shape = read_ml_1m()

In [3]:
ratings.head()

,user,item,rating,timestamp
0,0,0,5,978300760
1,0,1,3,978302109
2,0,2,3,978301968
3,0,3,4,978300275
4,0,4,5,978824291


In [4]:
item_data.head()

,item,title,genre
0,0,Toy Story (1995),Animation|Children's|Comedy
1,1,Jumanji (1995),Adventure|Children's|Fantasy
2,2,Grumpier Old Men (1995),Comedy|Romance
3,3,Waiting to Exhale (1995),Comedy|Drama
4,4,Father of the Bride Part II (1995),Comedy


In [5]:
train_set, test_set = split_recsys_data(ratings, test_size=0.1)
train_set.head()

,user,item,rating,timestamp
0,6039,669,4,956703932
1,6039,323,4,956703954
2,6039,128,5,956703954
3,6039,41,4,956703977
4,6039,1092,5,956703977


In [6]:
test_set.head()

,user,item,rating,timestamp
0,24,1315,3,978133433
1,24,1316,2,978133433
2,24,1223,3,978133460
3,24,1317,4,978133460
4,23,538,2,978133665


In [7]:
folds = split_by_user_folds(test_set)

In [72]:
from implicit.bpr import BayesianPersonalizedRanking
from implicit.evaluation import ndcg_at_k, mean_average_precision_at_k, precision_at_k

import scipy.sparse as sp

In [73]:
train_matrix = sp.csr_matrix(
    (
        (train_set.rating > 0).astype('float32'),
        (train_set.item, train_set.item)
    ),
    dtype='float32',
    shape=(shape[1], shape[0])
    
)

test_matrix = sp.csr_matrix(
    (
        (test_set.rating > 0).astype('float32'),
        (test_set.item, test_set.user)
    ),
    dtype='float32',
    shape=(shape[1], shape[0])
)

In [74]:
model = BayesianPersonalizedRanking(factors=50)

In [75]:
model.fit(train_matrix)

In [76]:
ndcg_at_k(model, train_matrix.T.tocsr(), test_matrix.T.tocsr(), 10)

0.028830271337937873

In [77]:
BayesianPersonalizedRanking??